In [1]:
import urllib
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time

In [2]:
from selenium import webdriver

# colab 환경이기 때문에 필요한 코드들
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--lang=ko_KR')

chrome_path = 'C:/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)

# file download path setting
prefs = {'download.default_directory' : '/content/drive/MyDrive/Colab Notebooks/마이크라우드/mycrowd_imgs'}
chrome_options.add_experimental_option('prefs', prefs)

C:\Users\ooche\AppData\Local\Temp/ipykernel_5208/3699737783.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)


In [3]:
# 마이크라우드 사이트 로그인 & 작업목록 접근

# driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'http://www.mycrowd.ai/login'
driver.get(url)

id = driver.find_element_by_id('id')
id.send_keys('***')  # 아이디

pw = driver.find_element_by_id('pwd')
pw.send_keys('***')  # 패스워드

driver.find_element_by_xpath('//*[@id="frm"]/fieldset/button').click()
time.sleep(1)

# driver.save_screenshot('login_test.png')

# 중복 로그인일 경우 (다른 환경에서 이미 로그인 되어 있는 상태인 경우) 아래 코드 주석 해제 후 실행
driver.find_element_by_xpath('/html/body/div/div/main/div/button[1]').click()
time.sleep(1)

driver.find_element_by_xpath('//*[@id="leftMenu"]/li[2]/ul/li[2]/a').click()
time.sleep(2)


# driver.save_screenshot('worklist_test.png')

In [4]:
##### annotation 에서 페이지 넘어가는 부분 오류

In [5]:
# Annotation task

project_list = []

for n in range(1, 100): # 페이지 넘기기
  try:
    if n == 1:
      pass
    elif n%11 == 1:
      continue
    elif n%11 != 0:
      driver.find_element_by_xpath(f'//*[@id="div_page"]/a[{n%11+1}]').click()
      time.sleep(1)
    elif n%11 == 0 :
      driver.find_element_by_xpath('//*[@id="div_page"]/a[@class="page-next on"]').click()
      time.sleep(1)

    soup = bs(driver.page_source, 'html.parser')
    projects = soup.select('div.card-list .task-card')

    for project in projects:
      temp = []

      task_level = project.select_one('.task-level')['data-task-level']
      task_code = project.select_one('.task-code').text
      task_title = project.select_one('.task-title').text
      task_info = project.select('.task-info span')[0].text
      reward = project.select_one('.reward').text

      temp.append(task_level)
      temp.append(task_code)
      temp.append(task_title)
      temp.append(task_info)
      temp.append(reward)

      project_list.append(temp)
      
    # 상세페이지 접근
    tasks = driver.find_elements_by_css_selector('#card_list > ul > li')

    for task in tasks:
      try: # 접근 가능 작업
        task.click()
        time.sleep(1)
        # driver.save_screenshot('task.png')
      except: # 접근 불가능 작업
        continue

      # 작업가이드(이미지 or PDF) 저장
      try:
        driver.find_element_by_class_name('file-down').click()
        time.sleep(1)
        driver.find_element_by_class_name('btn-pop-close').click()
      except:
        driver.find_element_by_xpath(".//html").send_keys(Keys.ENTER)
        time.sleep(1)

  except:
    print('수집 완료')
    break

print(project_list)
print(len(project_list))

수집 완료
[['3', 'P21E1-T6C83', '자율주행_데이터가공_10차_차선_075', '178 / 200', '200P'], ['3', 'P21E1-T6C19', '자율주행_데이터가공_10차_차선_063', '133 / 200', '200P'], ['3', 'P21E1-T6CEA', '자율주행_데이터가공_10차_차선_078', '120 / 200', '200P'], ['3', 'P21E1-T6C15', '자율주행_데이터가공_10차_차선_052', '102 / 200', '200P'], ['3', 'P21E1-T6C26', '자율주행_데이터가공_10차_차선_074', '53 / 200', '200P'], ['1', 'P2208-T6BD1', '(Polygon) 주행환경가공_007', '30 / 100', '1500P'], ['3', 'P21E1-T6C18', '자율주행_데이터가공_10차_차선_062', '23 / 200', '200P'], ['1', 'P2208-T6BAB', '(Polygon) 주행환경가공_062', '17 / 100', '1500P'], ['1', 'P2208-T6BAE', '(Polygon) 주행환경가공_064', '16 / 100', '1500P'], ['1', 'P22C6-T6D75', '박리_불량_RGB_3_111802', '14 / 194', '500P'], ['1', 'P2208-T6BCE', '(Polygon) 주행환경가공_006', '11 / 100', '1500P'], ['1', 'P22C6-T6D70', '박리_불량_RGB_2_111802', '9 / 163', '500P'], ['1', 'P22C6-T6D68', '박리_보통_RGB_1_110802', '6 / 57', '300P'], ['1', 'P22C7-T6D52', '철근노출_불량_RGB_1_112901', '4 / 373', '250P'], ['1', 'P22C7-T6D4C', '철근노출_불량_RGB_2_112601', '3 / 386', '250P'], 

In [6]:
# import pandas as pd

# df = pd.DataFrame(project_list)
# df.columns = ['level', 'task_code', 'title', 'info', 'reward']
# df

In [7]:
# from datetime import datetime

# current_time = datetime.now()

# df.to_csv('/content/drive/MyDrive/Colab Notebooks/마이크라우드/mycrowd_annotation_' + str(current_time) + '.csv', index=False, encoding='utf-8-sig')

In [8]:
# # Collection task

# driver.find_element_by_xpath('//*[@id="collect"]').click()

# nav = driver.find_element_by_id('div_page')
# pages = nav.find_elements_by_tag_name('a')

# project_list = []

# for page in range(1, len(pages) - 1):
#   driver.find_element_by_xpath(f'//*[@id="div_page"]/a[{page+1}]').click()
#   time.sleep(1)

#   soup = bs(driver.page_source, 'html.parser')
#   projects = soup.select('div.card-list .task-card')

#   for project in projects:
#     temp = []

#     task_code = project.select_one('.task-code').text
#     task_title = project.select_one('.task-title').text
#     task_info = project.select('.task-info span')[0].text
#     reward = project.select_one('.reward').text
    
#     temp.append(task_code)
#     temp.append(task_title)
#     temp.append(task_info)
#     temp.append(reward)

#     project_list.append(temp)

# print(project_list)

In [9]:
# import pandas as pd

# df = pd.DataFrame(project_list)
# df.columns = ['task_code', 'title', 'info', 'reward']
# df

In [10]:
# from datetime import datetime

# current_time = datetime.now()

# df.to_csv('/content/drive/MyDrive/Colab Notebooks/마이크라우드/mycrowd_' + str(current_time) + '.csv', index=False, encoding='utf-8-sig')